In [ ]:
'''

'''

#!/usr/bin/env python
#-*- coding: utf-8 -*-
import sys
import os
import numpy as np
import pandas as pd
from scipy import signal
from random import randint

def main():
    threshold = 0.05 # trigger threshold - 5 percent of maximum of the pulse

    path = "/home/sommema4/git/SiPM_simulator/data/preproc/..."
    path_out = "/home/sommema4/git/SiPM_simulator/data/preproc"

    for filename in os.listdir(path):
        file_in = os.path.join(path, filename)
        if filename.endswith(".csv"):
            cut_off_freq, sample_freq = strip_filename(filename)
            df = read_csv(file_in)
            rows, columns = df.shape
            columns = columns - 2
            
            # Slice dataframe to the individual components of the signal
            total, light, afterpulse, crosstalk, dark_current = slice_dataframe(df, columns)
            
            # Optimize PSD techniques
            df_norm = amplitude_normalization(total, columns)
            df_n, df_p = mean_dataframe(df_norm)
            popt_ccm = ccm_optimization(df_n, df_p)
            popt_fga = fga_optimization(df_n, df_p)
            
            # Perform PSD techniques
            FOM_ccm = ccm(popt_ccm, total)
            FOM_fga = fga(popt_fga, total)
            FOM_caa = caa(total)
            
            plot_PSD_
            
            
        
def strip_filename(filename):
    base = os.path.basename(filename)
    name = os.path.splitext(base)[0]
    cut_off_freq, sample_freq = name.split("_")
    return cut_off_freq, sample_freq

def read_csv(filename):
    # Read data to pandas dataframe and transpose it
    df = pd.read_csv(filename, sep=' ')
    #df = df.iloc[:, :-1]
    return df

def slice_dataframe(df, columns):
    light = df.iloc[:, 0::4]
    afterpulse = df.iloc[:, 1::4]
    crosstalk = df.iloc[:, 2::4]
    dark_current = df.iloc[:, 3::4]
    total = pd.DataFrame(light[:, :columns].values + crosstalk[:, :columns].values + afterpulse[:, :columns].values + dark_current[:, :columns].values)
    total = pd.concat([total, light[:, columns:]], axis=1)
    return total, light, afterpulse, crosstalk, dark_current

def amplitude_normalization(df, columns):
    df = df[:, :columns].apply(lambda x : x / x.max(), axis=1)
    return df

def mean_dataframe(df):
    df_n = df[df['particle']=='n']
    df_p = df[df['particle']=='p']
    df_n = df_n.mean(axis=0)
    df_p = df_p.mean(axis=0)
    return df_n, df_p

def threshold_pulse(x):
    max_val = x.max()
    max_i = x.argmax()
    temp_beginning = np.abs(x[max_i:] - threshold)
    temp_end = np.abs(x[:max_i] - threshold)
    pulse_beginning = temp_beginning.argmin()
    pulse_end = temp_end.argmin()
    return pulse_beginning, pulse_end

def integral_pulse(x, start, stop):
    return x[start:stop].sum()

def max_pulse(x):
    return x.max(), x.argmax()

def ccm_optimization(x):
    n_start, n_stop = threshold_pulse(df_n)
    n_max, n_imax = max_pulse(df_n)
    p_start, p_stop = threshold_pulse(df_p)
    p_max, p_imax = max_pulse(df_p)
    n_integral = integral_pulse(x, n_start, n_stop)
    p_integral = integral_pulse(x, p_start, p_stop)
    temp = []
    for i in min([n_stop - n_imax, p_stop - p_imax]):
        n_tail = integral_pulse(x, n_imax + i, n_stop)
        p_tail = integral_pulse(x, p_imax + i, p_stop)
        temp.append(np.abs((n_tail / n_integral) - (p_tail / p_integral)))
    return temp.argmax()

def caa(df):
    

def fft(df_n, df_p):



        
main()